# 개와 고양이 이미지 분류

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf

# =============================================
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth( gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)        
# =============================================
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

# 데이터 로드

In [2]:
import os

In [3]:
# 훈련 이미지 데이터와 테스트 이미지 데이터 경로 설정
train_cats_dir = './Data/cats_and_dogs_filtered/train/cats'
train_dogs_dir = './Data/cats_and_dogs_filtered/train/dogs'
test_cats_dir  = './Data/cats_and_dogs_filtered/test/cats'
test_dogs_dir  = './Data/cats_and_dogs_filtered/test/dogs'
predict_data_dir  = './Data/cats_and_dogs_filtered/predict' # 예측 데이터

In [4]:
# 경로 내에 있는 파일 목록을 리스트로 생성한다.
train_cats_fnames = os.listdir( train_cats_dir)
print(train_cats_fnames[:5])
train_dogs_fnames = os.listdir( train_dogs_dir)
print(train_dogs_fnames[:5])
test_cats_fnames = os.listdir( test_cats_dir)
print(test_cats_fnames[:5])
test_dogs_fnames = os.listdir( test_dogs_dir)
print(test_dogs_fnames[:5])
predict_data_fnames = os.listdir( predict_data_dir) # 예측데이터 파일목록 가져오기
print(predict_data_fnames[:5])

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: './Data/cats_and_dogs_filtered/train/cats'

# 이미지 불러오기 (train 2000장, test 1000장)

In [ ]:
path = os.path.join( train_cats_dir , train_cats_fnames[0])
path

In [ ]:
Image.open(path)

In [ ]:
figure, axes = plt.subplots(nrows =2 , ncols = 5, figsize = (15,5))
im1 = os.path.join( train_cats_dir, train_cats_fnames[0])
axes[0][0].imshow( Image.open(im1) )
im2 = os.path.join( train_cats_dir, train_cats_fnames[1])
axes[0][1].imshow( Image.open(im2) )
im3 = os.path.join( train_cats_dir, train_cats_fnames[2])
axes[0][2].imshow( Image.open(im3) )
im4 = os.path.join( train_cats_dir, train_cats_fnames[3])
axes[0][3].imshow( Image.open(im4) )
im5 = os.path.join( train_cats_dir, train_cats_fnames[4])
axes[0][4].imshow( Image.open(im5) )

im6 = os.path.join( train_dogs_dir, train_dogs_fnames[0])
axes[1][0].imshow( Image.open(im6) )
im7 = os.path.join( train_dogs_dir, train_dogs_fnames[1])
axes[1][1].imshow( Image.open(im7) )
im8 = os.path.join( train_dogs_dir, train_dogs_fnames[2])
axes[1][2].imshow( Image.open(im8) )
im9 = os.path.join( train_dogs_dir, train_dogs_fnames[3])
axes[1][3].imshow( Image.open(im9) )
im10 = os.path.join( train_dogs_dir, train_dogs_fnames[4])
axes[1][4].imshow( Image.open(im10) )



# 이미지 사이즈를 동일 사이즈로 변환하기

In [ ]:
figure, axes = plt.subplots(nrows =2 , ncols = 5, figsize = (15,5))
im1 = os.path.join( train_cats_dir, train_cats_fnames[0])
axes[0][0].imshow( Image.open(im1).resize((224,224)) )
im2 = os.path.join( train_cats_dir, train_cats_fnames[1])
axes[0][1].imshow( Image.open(im2).resize((224,224)) )
im3 = os.path.join( train_cats_dir, train_cats_fnames[2])
axes[0][2].imshow( Image.open(im3).resize((224,224)) )
im4 = os.path.join( train_cats_dir, train_cats_fnames[3])
axes[0][3].imshow( Image.open(im4).resize((224,224)) )
im5 = os.path.join( train_cats_dir, train_cats_fnames[4])
axes[0][4].imshow( Image.open(im5).resize((224,224)) )

im6 = os.path.join( train_dogs_dir, train_dogs_fnames[0])
axes[1][0].imshow( Image.open(im6).resize((224,224)) )
im7 = os.path.join( train_dogs_dir, train_dogs_fnames[1])
axes[1][1].imshow( Image.open(im7).resize((224,224)) )
im8 = os.path.join( train_dogs_dir, train_dogs_fnames[2])
axes[1][2].imshow( Image.open(im8).resize((224,224)) )
im9 = os.path.join( train_dogs_dir, train_dogs_fnames[3])
axes[1][3].imshow( Image.open(im9).resize((224,224)) )
im10 = os.path.join( train_dogs_dir, train_dogs_fnames[4])
axes[1][4].imshow( Image.open(im10).resize((224,224)) )



In [ ]:
# 파일 경로와 파일이름, 리사이즈를 수행할 사이즈 정보를 넘겨서 
# 이미지 크기를 재 설정 한다.
def load_images( default_path, filenames, resize_shape):
    images = []
    
    for fname in filenames:
        # 파일 경로 조립
        filepath = os.path.join(default_path, fname) 
        # 이미지 크기 재설정
        image = Image.open(filepath).resize(resize_shape) 
        # 이미지를 numpy 배열로 변경
        images.append(np.array(image))               
    # 전체 이미지 리스트를 numpy 배열로 반환한다
    return np.array(images)              
    

In [ ]:
X_train_cats = load_images( train_cats_dir, train_cats_fnames,(224,224))
X_train_dogs = load_images( train_dogs_dir, train_dogs_fnames,(224,224))
X_test_cats  = load_images( test_cats_dir,  test_cats_fnames ,(224,224))
X_test_dogs  = load_images( test_dogs_dir,  test_dogs_fnames ,(224,224))
 # 예측데이터 균일사이즈로 변환
Predict_Data  = load_images( predict_data_dir,  predict_data_fnames ,(224,224))

In [ ]:
X_train_cats.shape, X_train_dogs.shape, X_test_cats.shape, X_test_dogs.shape, Predict_Data.shape

# 데이터 병합

In [ ]:
X_train = np.concatenate((X_train_cats, X_train_dogs))
X_test  = np.concatenate((X_test_cats , X_test_dogs))

X_train.shape , X_test.shape

# 레이블 설정( 고양이 : 0 , 개 : 1 )

In [ ]:
y_train = np.array( [0]*1000 + [1]*1000)
y_test  = np.array( [0]*500 + [1]*500)

# numpy zip 형태로 저장

In [ ]:
np.savez_compressed( './cat_dog',
                    X_train = X_train,
                    X_test = X_test,
                    y_train = y_train,
                    y_test = y_test)

# numpy zip  로딩

In [ ]:
data = np.load( './cat_dog.npz')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

X_train.shape, X_test.shape, y_train.shape, y_test.shape

# 검증 셋 생성

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# 훈련 데이터 셋을 8:2로 나누어 검증용 데이터 셋을 생성
X_train, X_val, y_train, y_val = train_test_split( X_train, 
                                                   y_train, 
                                                   test_size = 0.2)

# CNN 모델 생성

In [ ]:
model = Sequential()
#               필터(커널)수 , 필터(커널) 사이즈,  
model.add( Conv2D( 32 , kernel_size = (3,3),
                   input_shape = (224,224,3),
                   activation = 'relu',
                   padding = 'same'))
model.add( MaxPool2D( pool_size = 2))
# ===========================================
model.add( Conv2D( 64 , kernel_size = ( 3,3),
                   activation = 'relu',
                   padding = 'same'))
model.add( MaxPool2D( pool_size = 2))
# ===========================================
model.add( Conv2D( 128 , kernel_size = ( 3,3),
                   activation = 'relu',
                   padding = 'same'))
model.add( MaxPool2D( pool_size = 2))
# ===========================================
model.add( Conv2D( 64 , kernel_size = ( 3,3),
                   activation = 'relu',
                   padding = 'same'))
model.add( MaxPool2D( pool_size = 2))
# ===========================================
model.add( Flatten())
# ===========================================

# ================ MLP ======================
model.add(Dense( 512, activation= 'relu'))
model.add(Dense( 1, activation = 'sigmoid'))


# 모델 컴파일
#### tf.keras.optimizer.Adam(lr = 0.0001)

In [ ]:
model.compile( loss = 'binary_crossentropy',
               optimizer = tf.keras.optimizers.Adam(lr = 0.0001),
               metrics= ['accuracy'])

In [ ]:
model.summary()

In [ ]:
earlyStopping = EarlyStopping(monitor='val_accuracy',
                             patience=15)
path = "./Model/CNN_catsndogs_{epoch:03d}_{val_accuracy:.4f}.hdf5"

modelCheckPoint = ModelCheckpoint(filepath=path,
                                 monitor='val_accuracy',
                                 verbose=1,
                                 save_best_only=True)

In [ ]:
history = model.fit( X_train , y_train,
                     epochs = 50,
                     validation_data = ( X_val , y_val),
                     batch_size = 32,
                     callbacks = [earlyStopping, modelCheckPoint])

In [ ]:
final_model = load_model('./Model/CNN_catsndogs_004_0.7000.hdf5')

In [ ]:
final_model.evaluate( X_test, y_test)

# 과대적합 방지 ( 드롭아웃, 데이터 부풀리기 )

In [ ]:
model2 = Sequential()
# =================================================
model2.add(Conv2D( 32, kernel_size = (3,3),
                   input_shape = ( 224, 224, 3),
                   activation = 'relu',
                   padding = 'same'))
model2.add(Conv2D( 32, kernel_size = (3,3),
                  activation = 'relu',
                  padding = 'same'))
model2.add(Dropout(0.3))
model2.add(MaxPool2D( pool_size = 2))
# =================================================
model2.add(Conv2D( 64, kernel_size = (3,3),
                  activation = 'relu',
                  padding = 'same'))
model2.add(Conv2D( 64, kernel_size = (3,3),
                  activation = 'relu',
                  padding = 'same'))
model2.add(Dropout(0.3))
model2.add(MaxPool2D( pool_size = 2))
# =================================================
model2.add(Conv2D( 128, kernel_size = (3,3),
                  activation = 'relu',
                  padding = 'same'))
model2.add(Conv2D( 128, kernel_size = (3,3),
                  activation = 'relu',
                  padding = 'same'))
model2.add(MaxPool2D( pool_size = 2))
# =================================================
model2.add(Conv2D( 256, kernel_size = (3,3),
                  activation = 'relu',
                  padding = 'same'))
model2.add(Conv2D( 256, kernel_size = (3,3),
                  activation = 'relu',
                  padding = 'same'))
model2.add(MaxPool2D( pool_size = 2))
model2.add(Dropout(0.3))
# =================================================
model2.add(Flatten())

# =================== MLP =========================
model2.add(Dense(512, activation = 'relu'))
model2.add(Dropout(0.3))
model2.add(Dense( 1 , activation = 'sigmoid'))

# =================== Model Compile ===============
model2.compile( loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(lr = 0.0001),
                metrics = ['accuracy'] )

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_gen = ImageDataGenerator( rotation_range = 90,
                              width_shift_range = 0.1,
                              zoom_range = 0.3,
                              vertical_flip = True )

In [ ]:
val_img_set = img_gen.flow(X_val, y_val, batch_size = 16)

In [ ]:
model2.fit_generator( img_gen.flow( X_train, y_train , batch_size = 16),
                      steps_per_epoch = 100, 
                      epochs = 20,
                      validation_data= ( val_img_set))

In [ ]:
model2.evaluate( X_test, y_test)

In [ ]:
predict_data_set = img_gen.flow_from_directory('./Data/cats_and_dogs_filtered/predict',
                                              target_size = (224,224),
                                              batch_size = 16,
                                              class_mode ='binary')

In [ ]:
result_predict = model2.predict_generator( Predict_Data ) # 예측데이터 넣기
print(result_predict)

In [ ]:
#from sklearn.metrics import confusion_matrix
#print( confusion_matrix( result_predict , [1,1,1,1,1,0,0,0,0,0]))

In [ ]:
result_predict_class = model2.predict_classes( Predict_Data) # 분류값으로출력 
print(result_predict_class)